# Payoff Matrices (part 4)

> This module contains payoff matrices for different evolutionary games
>
> Part 4 contains payoff matrices for the following games
> - A public threat level model of AI races